<a href="https://colab.research.google.com/github/Tessellate-Imaging/Monk_Object_Detection/blob/master/application_model_zoo/Example%20-%20Caltech%20cameratrap%20ECCV%20animal%20detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About the network

1. Paper on Focal Loss: https://arxiv.org/abs/1708.02002

2. Blog 1 on RetinaNet: https://medium.com/@14prakash/the-intuition-behind-retinanet-eb636755607d

3. Blog 2 on RetinaNet: https://blog.zenggyu.com/en/post/2018-12-05/retinanet-explained-and-demystified/

4. Blog 3 on RetinaNet: https://towardsdatascience.com/review-retinanet-focal-loss-object-detection-38fba6afabe4

5. Blog 4 on RetinaNet: https://analyticsindiamag.com/what-is-retinanet-ssd-focal-loss/

6. Blog 5 on RetinaNet: https://towardsdatascience.com/object-detection-on-aerial-imagery-using-retinanet-626130ba2203

# Table of contents


## 1. Installattion Instructions


## 2. Use trained model to detect animals in cameratrap images


## 3. How to train using caltech-cameratrap dataset

# Installation

 - Run these commands
     
     - git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git
     
     - cd Monk_Object_Detection/5_pytorch_retinanet/installation
     
 - Select the right requirements file and run
 
     - cat requirements.txt | xargs -n 1 -L 1 pip install

In [ ]:
! git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git

In [ ]:
# For colab use the command below
! cd Monk_Object_Detection/5_pytorch_retinanet/installation && cat requirements_colab.txt | xargs -n 1 -L 1 pip install

# For Local systems and cloud select the other file
#! cd Monk_Object_Detection/5_pytorch_retinanet/installation && cat requirements.txt | xargs -n 1 -L 1 pip install

# Use already trained model for demo

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/5_pytorch_retinanet/lib/");

In [ ]:
from infer_detector import Infer

In [ ]:
gtf = Infer();

In [ ]:
# Download trained model

In [ ]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1X7o3lgyIp5Pn68Az0hvTfbqAH8TxKd1f' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1X7o3lgyIp5Pn68Az0hvTfbqAH8TxKd1f" -O obj_cameratrap_2_trained.zip && rm -rf /tmp/cookies.txt

In [ ]:
! unzip -qq obj_cameratrap_2_trained.zip

In [ ]:
gtf.Model(model_path="trained_model/final_model.pt");

In [ ]:
f = open("trained_model/classes.txt", 'r');
class_list = f.readlines();
f.close();
for i in range(len(class_list)):
    class_list[i] = class_list[i][:-1]

In [ ]:
class_list

In [ ]:
img_path = "trained_model/test/1.jpg";
scores, labels, boxes = gtf.Predict(img_path, class_list, vis_threshold=0.4);
from IPython.display import Image
Image(filename='output.jpg')

In [ ]:
img_path = "trained_model/test/2.jpg";
scores, labels, boxes = gtf.Predict(img_path, class_list, vis_threshold=0.4);
from IPython.display import Image
Image(filename='output.jpg')

In [ ]:
img_path = "trained_model/test/3.jpg";
scores, labels, boxes = gtf.Predict(img_path, class_list, vis_threshold=0.4);
from IPython.display import Image
Image(filename='output.jpg')

In [ ]:
img_path = "trained_model/test/4.jpg";
scores, labels, boxes = gtf.Predict(img_path, class_list, vis_threshold=0.4);
from IPython.display import Image
Image(filename='output.jpg')

In [ ]:
img_path = "trained_model/test/5.jpg";
scores, labels, boxes = gtf.Predict(img_path, class_list, vis_threshold=0.4);
from IPython.display import Image
Image(filename='output.jpg')

In [ ]:
img_path = "trained_model/test/6.jpg";
scores, labels, boxes = gtf.Predict(img_path, class_list, vis_threshold=0.4);
from IPython.display import Image
Image(filename='output.jpg')

# Train your own detector

## Dataset
    - Credits: https://beerys.github.io/CaltechCameraTraps/

In [ ]:
import os
import sys
import numpy as np
import json

In [ ]:
os.listdir("dataset")

In [ ]:
with open("dataset/instances_images.json") as f:
    data = json.load(f)

In [ ]:
data.keys()

In [ ]:
data["categories"]

In [ ]:
class_list = [];
for i in range(100):
    class_list.append("");
    
for i in range(len(data["categories"])):
    class_list[data["categories"][i]["id"]] = data["categories"][i]["name"];

In [ ]:
data["annotations"][0]

In [ ]:
len(data["annotations"]), len(os.listdir("dataset/images")), len(data["images"])

In [ ]:
from tqdm import tqdm

image_names = [];
bboxes = [];
ids = [];

for i in tqdm(range(len(data["annotations"]))):
    if('bbox' in data["annotations"][i].keys()):
        image_id = data["annotations"][i]["image_id"];
        bbox = data["annotations"][i]["bbox"];
        label = class_list[data["annotations"][i]["category_id"]];
        if(image_id not in image_names):
            image_names.append(image_id);
            bboxes.append([]);
            ids.append([]);
            

        index = image_names.index(image_id);
        bboxes[index].append(bbox)
        ids[index].append(label)

In [ ]:
len(image_names), len(bboxes)

In [ ]:
bboxes[1]

In [ ]:
combined = [];
for i in tqdm(range(len(image_names))):
    img_name = image_names[i] + ".jpg";
    wr= "";
    for j in range(len(bboxes[i])):
        x1 = int(bboxes[i][j][0]);
        y1 = int(bboxes[i][j][1]);
        w = int(bboxes[i][j][2]);
        h = int(bboxes[i][j][3]);
        x2 = x1 + w;
        y2 = y1 + h; 
        label = ids[i][j];
        
        wr += str(x1) + " " + str(y1) + " " + str(x2) + " " + str(y2) + " " + label + " ";
    
    wr = wr[:len(wr)-1];
    
    combined.append([img_name, wr])

In [ ]:
combined[:5]

In [ ]:
import pandas as pd
df = pd.DataFrame(combined, columns = ['Id', 'Label'])  

In [ ]:
df.to_csv("dataset/train_labels.csv", index=False)

In [ ]:
import os
import numpy as np 
import cv2
import dicttoxml
import xml.etree.ElementTree as ET
from xml.dom.minidom import parseString
from tqdm import tqdm
import shutil
import json
import pandas as pd

In [ ]:
root = "dataset/";
img_dir = "images/";
anno_file = "train_labels.csv";

dataset_path = root;
images_folder = root + "/" + img_dir;
annotations_path = root + "/annotations/";

if not os.path.isdir(annotations_path):
    os.mkdir(annotations_path)
    
input_images_folder = images_folder;
input_annotations_path = root + "/" + anno_file;

output_dataset_path = root;
output_image_folder = input_images_folder;
output_annotation_folder = annotations_path;

tmp = img_dir.replace("/", "");
output_annotation_file = output_annotation_folder + "/instances_" + tmp + ".json";
output_classes_file = output_annotation_folder + "/classes.txt";

if not os.path.isdir(output_annotation_folder):
    os.mkdir(output_annotation_folder);
    
df = pd.read_csv(input_annotations_path);
columns = df.columns

delimiter = " ";

list_dict = [];
anno = [];
for i in range(len(df)):
    img_name = df[columns[0]][i];
    labels = df[columns[1]][i];
    tmp = labels.split(delimiter);
    for j in range(len(tmp)//5):
        label = tmp[j*5+4];
        if(label not in anno):
            anno.append(label);
    anno = sorted(anno)
    
for i in tqdm(range(len(anno))):
    tmp = {};
    tmp["supercategory"] = "master";
    tmp["id"] = i;
    tmp["name"] = anno[i];
    list_dict.append(tmp);

anno_f = open(output_classes_file, 'w');
for i in range(len(anno)):
    anno_f.write(anno[i] + "\n");
anno_f.close();


coco_data = {};
coco_data["type"] = "instances";
coco_data["images"] = [];
coco_data["annotations"] = [];
coco_data["categories"] = list_dict;
image_id = 0;
annotation_id = 0;


for i in tqdm(range(len(df))):
    img_name = df[columns[0]][i];
    labels = df[columns[1]][i];
    tmp = labels.split(delimiter);
    image_in_path = input_images_folder + "/" + img_name;
    img = cv2.imread(image_in_path, 1);
    h, w, c = img.shape;

    images_tmp = {};
    images_tmp["file_name"] = img_name;
    images_tmp["height"] = h;
    images_tmp["width"] = w;
    images_tmp["id"] = image_id;
    coco_data["images"].append(images_tmp);
    

    for j in range(len(tmp)//5):
        x1 = int(tmp[j*5+0]);
        y1 = int(tmp[j*5+1]);
        x2 = int(tmp[j*5+2]);
        y2 = int(tmp[j*5+3]);
        label = tmp[j*5+4];
        annotations_tmp = {};
        annotations_tmp["id"] = annotation_id;
        annotation_id += 1;
        annotations_tmp["image_id"] = image_id;
        annotations_tmp["segmentation"] = [];
        annotations_tmp["ignore"] = 0;
        annotations_tmp["area"] = (x2-x1)*(y2-y1);
        annotations_tmp["iscrowd"] = 0;
        annotations_tmp["bbox"] = [x1, y1, x2-x1, y2-y1];
        annotations_tmp["category_id"] = anno.index(label);

        coco_data["annotations"].append(annotations_tmp)
    image_id += 1;

outfile =  open(output_annotation_file, 'w');
json_str = json.dumps(coco_data, indent=4);
outfile.write(json_str);
outfile.close();

# Train your own detector

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/5_pytorch_retinanet/lib/");

In [ ]:
from train_detector import Detector

In [ ]:
gtf = Detector();

In [ ]:
root_dir = "dataset/";
coco_dir = "";
img_dir = "";
set_dir = "images";

In [ ]:
gtf.Train_Dataset(root_dir, coco_dir, img_dir, set_dir, batch_size=16, use_gpu=True)

In [ ]:
gtf.Model(model_name="resnet34");

In [ ]:
gtf.Set_Hyperparams(lr=0.0001, val_interval=1, print_interval=1)

In [ ]:
gtf.Train(num_epochs=100, output_model_name="final_model.pt");

# Inference

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/5_pytorch_retinanet/lib/");

In [ ]:
from infer_detector import Infer

In [ ]:
gtf = Infer();

In [ ]:
gtf.Model(model_path="final_model.pt");

In [ ]:
f = open("dataset/annotations/classes.txt", 'r');
class_list = f.readlines();
f.close();
for i in range(len(class_list)):
    class_list[i] = class_list[i][:-1]

In [ ]:
lst = os.listdir("dataset/images/")

In [ ]:
img_path = "dataset/images/" + lst[0];
scores, labels, boxes = gtf.Predict(img_path, class_list, vis_threshold=0.4);
from IPython.display import Image
Image(filename='output.jpg')

In [ ]:
img_path = "dataset/images/" + lst[1];
scores, labels, boxes = gtf.Predict(img_path, class_list, vis_threshold=0.4);
from IPython.display import Image
Image(filename='output.jpg')